In [11]:
import kagglehub
import os
import pandas as pd
# import UFC stats (individual figthers)
# Download latest version
data_path_set = '/Users/samuelcuriel/.cache/kagglehub/datasets/asaniczka/ufc-fighters-statistics/versions/16'
# Load the data from the csv and read it 
# to a dataframe
files = os.listdir(data_path_set)
print("files in directory: ", files)
file_path = os.path.join(data_path_set, 'ufc-fighters-statistics.csv')
df = pd.read_csv(file_path)
print(df.head())


files in directory:  ['ufc-fighters-statistics.csv']
                name    nickname  wins  losses  draws  height_cm  \
0    Robert Drysdale         NaN     7       0      0     190.50   
1  Daniel McWilliams  The Animal    15      37      0     185.42   
2         Dan Molina         NaN    13       9      0     177.80   
3          Paul Ruiz         NaN     7       4      0     167.64   
4    Collin Huckbody      All In     8       2      0     190.50   

   weight_in_kg  reach_in_cm    stance date_of_birth  \
0         92.99          NaN  Orthodox    1981-10-05   
1         83.91          NaN       NaN           NaN   
2         97.98          NaN       NaN           NaN   
3         61.23          NaN       NaN           NaN   
4         83.91       193.04  Orthodox    1994-09-29   

   significant_strikes_landed_per_minute  significant_striking_accuracy  \
0                                   0.00                            0.0   
1                                   3.36           